## Setup & Data

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import tensorflow_datasets as tfds
from typing import Dict, Optional, Tuple
from gpflow.utilities import to_default_float
import gpflow
import numpy as np
import deep_ckern as dkern

gpflow.config.set_default_float(np.float32)

Import MNIST and preprocess

In [2]:
original_dataset, info = tfds.load(name="mnist", split=tfds.Split.TRAIN, with_info=True)

total_num_data = info.splits["train"].num_examples
image_shape = info.features["image"].shape
image_size = tf.reduce_prod(image_shape)
num_classes = info.features["label"].num_classes
batch_size = 32

# reshape each input x to [1, 28, 28] and normalise to (0, 1).
# one-hot encode the outputs to [1, 10].
# x and y have a leading dim of 1 so that the concat in the next cell works
def map_fn(input_slice: Dict[str, tf.Tensor]):
    updated = input_slice
    image = to_default_float(updated["image"]) / 255.0
    label = tf.expand_dims(tf.one_hot(updated["label"], num_classes), 0)
    return tf.reshape(image, [-1, 28, 28]), label

n_train, n_test = 100, 50
n_total = n_train + n_test

autotune = tf.data.experimental.AUTOTUNE
dataset = (
    original_dataset.shuffle(1024)
    .take(n_total)
    # .batch(batch_size, drop_remainder=True)
    .map(map_fn, num_parallel_calls=autotune)
    .prefetch(autotune)
    # .repeat()
)

2021-09-28 15:26:37.185036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 15:26:37.190222: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 15:26:37.190533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-28 15:26:37.191390: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Turn the dataset into usable tensors

In [3]:
X = []
Y = []
for (x, y) in dataset:
    X.append(x)
    Y.append(y)
X = tf.concat(X, 0)
Y = tf.concat(Y, 0)
X_train, X_test = X[:n_train], X[n_train:]
Y_train, Y_test = Y[:n_train], Y[n_train:]

2021-09-28 15:26:37.787361: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-28 15:26:37.858571: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Run the model

In [4]:
def create_deep_kern():
    return dkern.DeepKernel(
        [1, 28, 28],
        filter_sizes=[[5, 5], [2, 2], [5, 5], [2, 2]],
        recurse_kern=dkern.ExReLU(multiply_by_sqrt2=True),
        var_weight=1.,
        var_bias=1.,
        padding=["VALID", "SAME", "VALID", "SAME"],
        strides=[[1, 1]] * 4,
        data_format="NCHW",
        skip_freq=-1,
    )

deep_kern = create_deep_kern()

Compute the training covariance $K_{xx}$ and train-test cross covariance $K_{zx}$:

In [5]:
Kxx = deep_kern.K(X_train)
Kzx = deep_kern.K(X_test, X_train)

2021-09-28 15:26:38.175450: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


Treat the classification as a multi-output regression problem (with independent outputs) and solve for the predictive means:

In [6]:
K_inv_Y = tf.linalg.solve(Kxx, Y_train)
pred_means = tf.linalg.matmul(Kzx, K_inv_Y)

Y_pred = tf.one_hot(tf.math.argmax(pred_means, axis=1), 10)
corrects = tf.reduce_all(Y_pred == Y_test, axis=1)
accuracy = (tf.math.count_nonzero(corrects) / len(corrects)).numpy()

print(accuracy) # 88%

0.88


2021-09-28 15:26:38.946831: I tensorflow/core/util/cuda_solvers.cc:180] Creating CudaSolver handles for stream 0x55ac717c1c40


TODO: ResNet kernel

In [7]:
def create_res_kern():
    depth = 32
    n_blocks = 3
    
    # Copied from https://github.com/tensorflow/models/blob/d089975f/official/resnet/cifar10_main.py#L167-L170
    if depth % 6 != 2:
        raise ValueError('DEPTH must be 6n + 2:', depth)
    block_depth = (depth - 2) // 6
    
    return dkern.resnet.ResnetKernel(
        input_shape=[1, 28, 28],
        block_sizes=[block_depth]*n_blocks,
        block_strides=[1, 2, 2, 2, 2, 2, 2][:n_blocks],
        kernel_size=3,
        conv_stride=1,
        recurse_kern=dkern.ExReLU(),
        var_weight=1.,  # scipy.stats.truncnorm.std(a=-2, b=2, loc=0., scale=1.)**2,
        var_bias=1.,
        data_format='NCHW',
    )
    
res_kern = create_res_kern()

In [8]:
Kxx_res = res_kern.K(X_train)
Kzx_res = res_kern.K(X_test, X_train)

initial_conv 1
Tensor("inputs:0", shape=(10100, 1, 28, 28), dtype=float32)
projection_shortcut Tensor("strides:0", shape=(), dtype=int32)
Tensor("cond/Identity:0", shape=(10100, 1, 28, 28), dtype=float32)


OperatorNotAllowedInGraphError: in user code:

    /home/ross/prog/convnets-as-gps/deep_ckern/resnet.py:58 conv2d_fixed_padding  *
        data_format=data_format)
    /home/ross/prog/convnets-as-gps/.venv/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /home/ross/prog/convnets-as-gps/.venv/lib/python3.9/site-packages/tensorflow/python/ops/nn_ops.py:2281 conv2d_v2
        return conv2d(input,  # pylint: disable=redefined-builtin
    /home/ross/prog/convnets-as-gps/.venv/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:206 wrapper
        return target(*args, **kwargs)
    /home/ross/prog/convnets-as-gps/.venv/lib/python3.9/site-packages/tensorflow/python/ops/nn_ops.py:2371 conv2d
        padding, explicit_paddings = convert_padding(padding)
    /home/ross/prog/convnets-as-gps/.venv/lib/python3.9/site-packages/tensorflow/python/ops/nn_ops.py:1890 convert_padding
        if padding == "EXPLICIT":
    